# Initialise Client API

Indicate the project used for making query

In [1]:
from google.cloud import bigquery
import pandas as pd   # for parsing query into dataframe
client = bigquery.Client(project="treenyc-384104") # mark the created project id

# Retrieve Schema

Retrieve schemas for each tables to check compatability.

In [2]:
# get tables name inside dataset
table_names = [table.table_id for table in client.list_tables("bigquery-public-data.new_york_trees")]
print(table_names)

['tree_census_1995', 'tree_census_2005', 'tree_census_2015', 'tree_species']


In [30]:
descrption_dfs = {}
for table in client.list_tables("bigquery-public-data.new_york_trees"):
    table_schema = client.get_table(table).schema
    table_metadata = pd.DataFrame(
        {
            "Name" : [item.name for item in table_schema],
            "Dtype" : [item.field_type for item in table_schema],
            "Example" : client.list_rows(table=table, max_results=1).to_dataframe().values.flatten(),
            "Description":[item.description for item in table_schema],
        }
    )
    with open(f"meta/{table.table_id}_meta.txt", 'w') as f:
        f.write(table_metadata.to_markdown())
        f.close()
    descrption_dfs[table.table_id] = table_metadata
print(descrption_dfs)

{'tree_census_1995':                   Name    Dtype            Example   
0             recordid  INTEGER              21111  \
1              address   STRING       700 W 123 ST   
2         house_number   STRING              700.0   
3               street   STRING           W 123 ST   
4         zip_original  INTEGER                  0   
5          cb_original  INTEGER                  0   
6                 site   STRING             Across   
7              species   STRING               QUPA   
8             diameter  INTEGER                 15   
9               status   STRING               Poor   
10               wires   STRING               None   
11  sidewalk_condition   STRING               Good   
12   support_structure   STRING               None   
13             borough   STRING          Manhattan   
14                   x    FLOAT                1.0   
15                   y    FLOAT                1.0   
16           longitude    FLOAT                1.0   
17     

# Query

Write the query using SQL

In [48]:
QUERY = (   # query in SQL style
    """
    SELECT *
    FROM `bigquery-public-data.new_york_trees.tree_census_1995`
    LIMIT 10
    """
)

Run the main query and get as a pandas dataframe

In [49]:
query = client.query(QUERY) # create query
df = query.to_dataframe()   # convert into pandas dataframe
df

,recordid,address,house_number,street,zip_original,cb_original,site,species,diameter,status,...,latitude,cb_new,zip_new,censustract_2010,censusblock_2010,nta_2010,segmentid,spc_common,spc_latin,location
0,21111,700 W 123 ST,700.0,W 123 ST,0,0,Across,QUPA,15,Poor,...,1.0,0,0,None,None,Unknown,0,"OAK, PIN",QUERCUS PALUSTRIS,"(1.0, 1.0)"
1,21116,700 W 123 ST,700.0,W 123 ST,0,0,Across,QUPA,13,Good,...,1.0,0,0,None,None,Unknown,0,"OAK, PIN",QUERCUS PALUSTRIS,"(1.0, 1.0)"
2,3219,53-054 WASHINGTON SQ W,None,WASHINGTON SQ W,0,0,Across,QUPA,7,Good,...,1.0,0,0,None,None,Unknown,0,"OAK, PIN",QUERCUS PALUSTRIS,"(1.0, 1.0)"
3,32660,870/860 1 AV,None,1 AV,0,0,Across,QUPH,11,Good,...,1.0,0,0,None,None,Unknown,0,"OAK, WILLOW",QUERCUS PHELLOS,"(1.0, 1.0)"
4,32661,870/860 1 AV,None,1 AV,0,0,Across,QUPH,11,Good,...,1.0,0,0,None,None,Unknown,0,"OAK, WILLOW",QUERCUS PHELLOS,"(1.0, 1.0)"
5,32662,870/860 1 AV,None,1 AV,0,0,Across,QUPH,6,Good,...,1.0,0,0,None,None,Unknown,0,"OAK, WILLOW",QUERCUS PHELLOS,"(1.0, 1.0)"
6,32663,870/860 1 AV,None,1 AV,0,0,Across,QUPH,6,Good,...,1.0,0,0,None,None,Unknown,0,"OAK, WILLOW",QUERCUS PHELLOS,"(1.0, 1.0)"
7,3570,767 E 58 ST,767.0,E 58 ST,0,0,Front,GLTR,10,Excellent,...,1.0,0,0,None,None,Unknown,0,HONEYLOCUST,GLEDITSIA TRIACANTHOS,"(1.0, 1.0)"
8,3571,767 E 58 ST,767.0,E 58 ST,0,0,Front,GLTR,9,Excellent,...,1.0,0,0,None,None,Unknown,0,HONEYLOCUST,GLEDITSIA TRIACANTHOS,"(1.0, 1.0)"
9,32664,870/860 1 AV,None,1 AV,0,0,Across,QUPH,6,Good,...,1.0,0,0,None,None,Unknown,0,"OAK, WILLOW",QUERCUS PHELLOS,"(1.0, 1.0)"
